In [1]:
import cv2
import numpy as np

net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

with open("coco.names", "r") as f:
    classes = f.read().strip().split("\n")

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*"XVID")
out = cv2.VideoWriter("output_video.avi", fourcc, fps, (width, height))

while True:
    ret, frame = cap.read()

    if not ret:
        print("Error: Could not read frame from the camera.")
        break

    height, width = frame.shape[:2]

    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)

    net.setInput(blob)

    output_layer_names = net.getUnconnectedOutLayersNames()

    outputs = net.forward(output_layer_names)

    object_boxes = []
    object_confidences = []
    object_labels = []

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5: 
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

               
                object_boxes.append((x, y, x + w, y + h))
                object_confidences.append(confidence)
                object_labels.append(classes[class_id])

   
    for i in range(len(object_boxes)):
        x, y, x_end, y_end = object_boxes[i]
        confidence = object_confidences[i]
        label = object_labels[i]

      
        color = (0, 255, 0)  #  square green
        cv2.rectangle(frame, (x, y), (x_end, y_end), color, 2)

        
        label_text = f"{label}: {confidence:.2f}"
        cv2.putText(frame, label_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

   
    out.write(frame)

   
    cv2.imshow("Object Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()

cv2.destroyAllWindows()


KeyboardInterrupt: 